In [ ]:
#| default_exp pheno

In [ ]:
#| export
import jax
import jax.numpy as jnp
from chewc.structs import Population, Trait, _flatten_gather_chr_locus
from typing import Tuple

# compute_dosage remains the same...
@jax.jit
def compute_dosage(population: Population) -> jnp.ndarray:
    """Computes allele dosage. Shape: (n_ind, n_chr, n_loci)"""
    return jnp.sum(population.geno, axis=2)




@jax.jit
def calculate_phenotypes(
    key: jax.Array,
    population: Population,
    trait: Trait,
    heritability: jnp.ndarray,  # (n_traits,)
) -> Tuple[jnp.ndarray, jnp.ndarray]:
    """Return (phenotypes, true_breeding_values)."""
    # Dosage per locus (0-2) as float
    dosage = jnp.sum(population.geno, axis=2, dtype=jnp.int32)  # (n, chr, loci)
    qtl_dosage = _flatten_gather_chr_locus(
        dosage, trait.qtl_chromosome, trait.qtl_position
    ).astype(jnp.float32)  # (n, n_qtl)

    tbv = trait.intercept + qtl_dosage @ trait.qtl_effects  # (n, n_traits)
    add_var = jnp.var(tbv, axis=0)  # (n_traits,)

    h2 = jnp.clip(heritability.astype(jnp.float32), 1e-8, 1.0 - 1e-8)
    env_var = jnp.maximum(0.0, add_var * ((1.0 / h2) - 1.0))
    noise = jax.random.normal(key, tbv.shape)
    phenotypes = tbv + noise * jnp.sqrt(env_var)

    return phenotypes, tbv


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()